# Preprocessing Pandas

In [ ]:
import pandas as pd
sample_submission = pd.read_csv("understanding_cloud_organization/sample_submission.csv")
train = pd.read_csv("understanding_cloud_organization/train_320.csv")

In [ ]:
train['Image_name'] = train['Image_Label'].apply(lambda x: x.split('_')[0])
train['Label_name'] = train['Image_Label'].apply(lambda x: x.split('_')[1])
train.drop('Image_Label',axis=1,inplace=True)
train = train.pivot('Image_name','Label_name','EncodedPixels')

In [ ]:
train.head()

# Affichage Image

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

img=mpimg.imread('./understanding_cloud_organization/train_images_320/009e2f3.jpg')
img2=mpimg.imread('./understanding_cloud_organization/train_images_320_Black_Fish_Gravel_Sugar_Flower/009e2f3.jpg')


f, axarr = plt.subplots(1,2)
axarr[0].imshow(img)
axarr[1].imshow(img2)



# DATA LOADING

In [ ]:
from transformations import MoveAxis, Normalize01

In [ ]:
from transformations import Compose, DenseTarget
from transformations import MoveAxis, Normalize01
from customdatasets2 import SegmentationDataSet
from torch.utils.data import DataLoader
from customdatasets import make_mask
from skimage.io import imread
from sklearn.model_selection import train_test_split
import pathlib
import numpy as np
import pandas as pd

df = pd.read_csv("understanding_cloud_organization/train_320.csv")

# root directory
root = pathlib.Path.cwd() / 'understanding_cloud_organization'
def get_filenames_of_path(path: pathlib.Path, ext: str = '*'):
    """Returns a list of files in a directory/path. Uses pathlib."""
    filenames = [file for file in path.glob(ext) if file.is_file()]
    return filenames

# input and target files
inputs = get_filenames_of_path(root / 'train_images_320')  
true_inputs = []
true_targets = []

shape = (320, 480)


for input in inputs:
    img_name = str(input).split('/')[-1]
    array_input = imread(input)
    true_inputs.append(array_input)
  #  true_inputs.append(np.fliplr(array_input))
 #   true_inputs.append(np.flipud(array_input))
    mask = make_mask(df, img_name, shape)
    true_targets.append(mask)
 #   true_targets.append(np.fliplr(mask))
  #  true_targets.append(np.flipud(mask))
    
# training transformations and augmentations
transforms = Compose([
    DenseTarget(),
    Normalize01(),
    MoveAxis(),
])

# random seed
random_seed = 42

# split dataset into training set and validation set
train_size = 0.8  # 80:20 split

inputs_train, inputs_valid = train_test_split(
    true_inputs,
    random_state=random_seed,
    train_size=train_size,
    shuffle=True)


targets_train, targets_valid = train_test_split(
    true_targets,
    random_state=random_seed,
    train_size=train_size,
    shuffle=True)

# dataset training
dataset_train = SegmentationDataSet(inputs=inputs_train,
                                    targets=targets_train,
                                    transform=transforms)

    

# dataset validation
dataset_valid = SegmentationDataSet(inputs=inputs_valid,
                                    targets=targets_valid,
                                    transform=transforms)


# dataloader training
dataloader_training = DataLoader(dataset=dataset_train,
                                 batch_size=4,
                                 shuffle=False)
# dataloader validation
dataloader_validation = DataLoader(dataset=dataset_valid,
                                   batch_size=4,
                                   shuffle=False)

dataloaders = {
    'train': dataloader_training,
    'val': dataloader_validation
} 


In [ ]:
x, y = next(iter(dataloader_training))
w, z = next(iter(dataloader_training))
print(f'x = shape: {x.shape}; type: {x.dtype}')
print(f'x = min: {x.min()}; max: {x.max()}')
print(f'y = shape: {y.shape}; class: {y.unique()}; type: {y.dtype}')
print(f'y = min: {y.min()}; max: {y.max()}')


In [ ]:
import torchvision.utils
from skimage.io import imread
import numpy as np
import helper
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

x, y = next(iter(dataloader_training))

true_inputs = x

true_labels = y
print(true_labels[0].numpy().shape)

def reverse_transform(inp):
    inp = inp.numpy()
    inp = np.moveaxis(inp, -1, 0)
    inp = np.moveaxis(inp, -1, 0)
    input_asarray = np.clip(inp, 0, 1)
    input_asarray = (input_asarray * 255).astype(np.uint8)
    return input_asarray

# Change channel-order and make 3 channels for matplot
input_images_rgb = [reverse_transform(x) for x in true_inputs]

# Map each channel (i.e. class) to each color
target_masks_rgb = [helper.masks_to_colorimg(x.cpu(), "xD") for x in true_labels]
f, axarr = plt.subplots(1,2)
axarr[0].imshow(input_images_rgb[0])
axarr[1].imshow(target_masks_rgb[0])

## MODEL UNET

In [ ]:
import torch
from unet import *
model = UNet(in_channels=3,
             out_channels=4,
             n_blocks=4,
             start_filters=32,
             activation='relu',
             normalization='batch',
             conv_mode='same',
             dim=2).cuda()

x = torch.randn(size=(1, 3, 320, 480), dtype=torch.float32).cuda()

with torch.no_grad():
    out = model(x)

print(f'Out: {out.shape}')

In [ ]:
from torchsummary import summary
summary = summary(model, (3, 320, 480))



In [ ]:
shape = 1920


def compute_max_depth(shape, max_depth=10, print_out=True):
    shapes = []
    shapes.append(shape)
    for level in range(1, max_depth):
        if shape % 2 ** level == 0 and shape / 2 ** level > 1:
            shapes.append(shape / 2 ** level)
            if print_out:
                print(f'Level {level}: {shape / 2 ** level}')
        else:
            if print_out:
                print(f'Max-level: {level - 1}')
            break

    return shapes


out = compute_max_depth(shape, print_out=True, max_depth=10)


In [ ]:
low = 10
high = 512
depth = 8


def compute_possible_shapes(low, high, depth):
    possible_shapes = {}
    for shape in range(low, high + 1):
        shapes = compute_max_depth(shape,
                                   max_depth=depth,
                                   print_out=False)
        if len(shapes) == depth:
            possible_shapes[shape] = shapes

    return possible_shapes
possible_shapes = compute_possible_shapes(low, high, depth)
possible_shapes


# TRAIN ET RUN

In [ ]:
from train import Trainer

# device
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    torch.device('cpu')

# criterion
criterion = torch.nn.BCEWithLogitsLoss()

# optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.002)

# trainer
trainer = Trainer(model=model,
                  device=device,
                  criterion=criterion,
                  optimizer=optimizer,
                  training_DataLoader=dataloader_training,
                  validation_DataLoader=dataloader_validation,
                  lr_scheduler=None,
                  epochs=2,
                  epoch=0,
                  notebook=True)

# start training
training_losses, validation_losses, lr_rates = trainer.run_trainer()


# SAVE THE MODEL

In [ ]:
model_name =  'cloud.pt'
torch.save(model.state_dict(), pathlib.Path.cwd() / model_name)

# LEARNING RATE

In [ ]:
from lr_rate_finder import *
lrf = LearningRateFinder(model, criterion, optimizer, device)
lrf.fit(dataloader_training, steps=1000)

In [ ]:
lrf.plot()


In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
fig = plt.show(training_losses, validation_losses)
print(training_losses)
print(validation_losses)

In [ ]:
from visual import plot_training
fig = plot_training(training_losses, validation_losses, lr_rates, gaussian=True, sigma=1, figsize=(10, 4))

In [ ]:
import math
from sklearn.model_selection import train_test_split

import pathlib

random_seed = 42

# root directory
root = pathlib.Path.cwd() / 'understanding_cloud_organization'
def get_filenames_of_path(path: pathlib.Path, ext: str = '*'):
    """Returns a list of files in a directory/path. Uses pathlib."""
    filenames = [file for file in path.glob(ext) if file.is_file()]
    return filenames

# input and target files
inputs = get_filenames_of_path(root / 'train_images_320')
targets = get_filenames_of_path(root / 'train_images_320_Black_Fish_Gravel_Sugar_Flower')

model.eval()   # Set model to evaluate mode

train_size = 0.8  # 80:20 split
inputs_train, inputs_valid = train_test_split(
    inputs,
    random_state=random_seed,
    train_size=train_size,
    shuffle=True)

test_dataset = SegmentationDataSet(inputs=inputs_train,
                                    targets=targets_train,
                                    transform=transforms)

test_loader = SegmentationDataSet(inputs=inputs_train,
                                    targets=targets_train,
                                    transform=transforms)

# dataloader training
dataloader_training = DataLoader(dataset=dataset_train,
                                 batch_size=2,
                                 shuffle=False)
# dataloader validation
dataloader_validation = DataLoader(dataset=dataset_valid,
                                   batch_size=2,
                                   shuffle=False)
        
x, y = next(iter(dataloader_training))
true_inputs = x.to(device)
true_labels = y.to(device)

pred = model(true_inputs).cpu()
out_pred = torch.softmax(pred, dim=1).detach()


true_inputs = true_inputs.cpu()
true_labels = true_labels.cpu()



import torchvision.utils
from skimage.io import imread
import numpy as np
import helper

def reverse_transform(inp):
    inp = inp.numpy()
    inp = np.moveaxis(inp, -1, 0)
    inp = np.moveaxis(inp, -1, 0)
    input_asarray = np.clip(inp, 0, 1)
    input_asarray = (input_asarray * 255).astype(np.uint8)
    return input_asarray

# Change channel-order and make 3 channels for matplot
input_images_rgb = [reverse_transform(x) for x in true_inputs]

# Map each channel (i.e. class) to each color
target_masks_rgb = [helper.masks_to_colorimg(x.cpu(), "xD") for x in true_labels]

pred_rgb = [helper.masks_to_colorimg(x, "xD") for x in out_pred]

f, axarr = plt.subplots(2,3)
axarr[0][0].imshow(input_images_rgb[0])
axarr[0][1].imshow(target_masks_rgb[0])
axarr[0][2].imshow(pred_rgb[0])
axarr[1][0].imshow(input_images_rgb[1])
axarr[1][1].imshow(target_masks_rgb[1])
axarr[1][2].imshow(pred_rgb[1])